In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install PyMuPDF sentence-transformers transformers scikit-learn openai

In [ ]:
import fitz
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from openai import OpenAI

In [ ]:
pdf_path = "/content/Human-Nutrition-2020.pdf"

In [ ]:
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

pdf_text = extract_text_from_pdf(pdf_path)

In [ ]:
def clean_text(text):
    text = text.replace('\n', ' ')
    text = ' '.join(text.split())
    return text

cleaned_text = clean_text(pdf_text)

In [ ]:
def chunk_text(text, chunk_size=1000, overlap=100):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunk = text[start:end]
        chunks.append(chunk)
        start += chunk_size - overlap
    return chunks

chunks = chunk_text(cleaned_text)
print(f"Total chunks: {len(chunks)}")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

tokens = tokenizer(
    chunks,
    truncation=True,
    padding='max_length',
    max_length=512,
    return_tensors='pt'
)

print(tokens['input_ids'].shape)

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(chunks, show_progress_bar=True)

In [ ]:
np.save('embeddings.npy', embeddings)

loaded_embeddings = np.load('embeddings.npy')
print(loaded_embeddings.shape)

query = input("Enter your question: ")
query_embedding = model.encode([query])



In [ ]:
similarities = cosine_similarity(query_embedding, loaded_embeddings)
top_k_idx = similarities[0].argsort()[-5:][::-1]

for i, idx in enumerate(top_k_idx):
    score = similarities[0][idx]
    chunk = chunks[idx]
    print(f"Chunk {i+1} (score: {score:.4f}): {chunk[:500]}...\n")

retrieved_chunks = [chunks[i] for i in top_k_idx]

context = "\n\n".join(retrieved_chunks)

prompt = f"""Answer the following question based on the provided context.
If the answer cannot be found in the context, say you don't know.

Question: {query}

Context: {context}
"""


In [ ]:
client = OpenAI(api_key="")

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ],
    temperature=0
)

print(response.choices[0].message.content)